# DATA622 Final Project
Daniel Dittenhafer

This notebook proceeds with using the trained convolutional neural network to predict emotion for each of the face images in the provided test set.

### Import References

In [1]:
import random
import numpy as np
import emotion_model
import dwdii_transforms

random.seed(20275)

Using Theano backend.


### Definitions

The model was trained on 150x150 images, therefore we will use this size of the test data as well.

In [2]:
imgDimension = []
imgDimension.append(150)
imgDimension.append(150)

### Load Kaggle Test Data

In [3]:
testDataPath = "../../facial_expressions/test"

In [4]:
testData, fileList = dwdii_transforms.load_test_data(testDataPath, imgResize=imgDimension, verboseFreq=20)

0.000000: y38.jpg
0.069930: 183a.jpg
0.139860: RamyaKrishna_71.jpg
0.209790: NA.NE2.200.jpg
0.279720: Brahmanandam_46.jpg
0.349650: 189a.jpg
0.419580: 89a.jpg
0.489510: 43a.jpg
0.559441: YM.SU2.59.jpg
0.629371: 37a.jpg
0.699301: y29.jpg
0.769231: 54b.jpg
0.839161: Brahmanandam_122.jpg
0.909091: Brahmanandam_24.jpg
0.979021: NM.SA2.99.jpg


In [5]:
testData.shape

(286, 150, 150)

In [6]:
testX = testData.reshape(testData.shape[0], 1, testData.shape[1],testData.shape[2])

### Define and Load Trained Model

In [7]:
# Map the emotions to integers for categorization later.
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

model = emotion_model.emotion_model_jh_v5(len(emotions), verbose=True, 
                                        input_shape=(1,imgDimension[0],imgDimension[1]))

{'sadness': 6, 'neutral': 2, 'contempt': 7, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
8
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2

In [8]:
loadWeights = True
if loadWeights:
    model.load_weights("dwdii-emo-150-jhv5a-Cloud.hdf5")

### Produce Predictions

In [9]:
predictOutput = model.predict(testX, batch_size=32, verbose=1)

286/286 [==============================] - 6s     


In [10]:
len(predictOutput)

286

In [12]:
print predictOutput[0]

[  4.27418536e-06   6.65761977e-12   1.35717960e-03   1.40786927e-09
   7.58282898e-04   5.20988049e-16   9.97880280e-01   9.49431271e-23]


In [13]:
np.array(predictOutput[0]).argmax()

6

In [14]:
numEmo = dwdii_transforms.numericEmotions()
numEmo[6]

'sadness'

In [15]:
fileList[0]

'y38.jpg'

## Persist Predictions

In this section we save the prediction results to a file in the prescribed Kaggle competition format.

In [ ]:
# For each prediction output, build the row for the CSV and add to our list for saving.
outputData = []
for i in range(len(predictOutput)):
    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predictedEmo = numEmo[predictionNdx]
    fileName = fileList[i]
    outputData.append([filename,predictionProb,predictedEmo])
   
with open('dwdii_predictions.csv', 'w') as mycsvfile:
    dw = csv.writer(mycsvfile)
    for row in outputData:
        dw.writerow(row)